File to train on the toy_df.csv (that came from the 001_Forbes Dataset)

In [12]:
import os
import ast
import torch
import random
import pandas as pd
from transformers import TapasConfig, TapasForQuestionAnswering, TapasTokenizer, AdamW

In [13]:
# load in tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
base_tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
# Function to convert the string representation to a list of tuples
def parse_answer_coords(coords_str):
    try:
        # Safely evaluate the string to a Python object
        coords = ast.literal_eval(coords_str)
        
        # Ensure the result is a list of tuples with integers
        if isinstance(coords, list) and all(
            isinstance(coord, (tuple, list)) and len(coord) == 2 and all(isinstance(x, int) for x in coord)
            for coord in coords
        ):
            return [tuple(coord) for coord in coords]  # Convert lists to tuples if needed
        else:
            raise ValueError(f"Invalid format for answer_coords: {coords_str}")
    except (ValueError, SyntaxError) as e:
        raise ValueError(f"Error parsing answer_coords: {coords_str}. Details: {e}")

In [15]:
# class to tokenizer dataset
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = self.process_answer_coords_column(data)
        self.tokenizer = tokenizer
        self.table_csv_path = 'data/'

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        table = pd.read_csv(self.table_csv_path + item.dataset).astype(
            str
        )  # be sure to make your table data text only
        encoding = self.tokenizer(
            table=table,
            queries=item.question,
            answer_coordinates=item.answer_coords,
            answer_text=item.sample_answer,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        # remove the batch dimension which the tokenizer adds by default
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        # add the float_answer which is also required (weak supervision for aggregation case)
        encoding["float_answer"] = torch.tensor(item.float_answer)
        return encoding

    def __len__(self):
        return len(self.data)
    
    # change answer_coords from strings
    def process_answer_coords_column(self, data, column_name="answer_coords"):
        if column_name not in data.columns:
            raise ValueError(f"Column '{column_name}' does not exist in the dataset.")

        def parse_answer_coords(coords_str):
            try:
                coords = ast.literal_eval(coords_str)
                if isinstance(coords, list) and all(
                    isinstance(coord, (tuple, list)) and len(coord) == 2 and all(isinstance(x, int) for x in coord)
                    for coord in coords
                ):
                    return [tuple(coord) for coord in coords]
                else:
                    raise ValueError(f"Invalid format for answer_coords: {coords_str}")
            except (ValueError, SyntaxError) as e:
                raise ValueError(f"Error parsing answer_coords: {coords_str}. Details: {e}")
        data[column_name] = data[column_name].apply(parse_answer_coords)
        return data

In [20]:
# load in toy_df, tokenize and place in dataloader
csv_path = 'data/toy_df_v2.csv'
train_df = pd.read_csv(csv_path)

# load train dataloader
train_dataset = TableDataset(train_df, tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8)

dev_datasets = ['040_Speed.cs]

# load dev dataloader
dev_dataset = TableDataset(toy_df_dev, tokenizer)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=8)

Dev Datasets: ['010_ECommerce.csv', '005_NYC.csv']
Train Datasets: ['004_Taxi.csv', '002_Titanic.csv', '001_Forbes.csv', '008_Tornados.csv', '011_SF.csv', '013_Roller.csv', '018_Staff.csv', '015_Food.csv', '009_Central.csv', '017_Hacker.csv', '016_Holiday.csv', '012_Heart.csv', '019_Aircraft.csv', '007_Fifa.csv']


/var/folders/f1/_zcz3_s54kz_1zxxwgs414pw0000gn/T/ipykernel_16293/1796820079.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].apply(parse_answer_coords)


In [21]:
# load in WTQ model (from TAPAS documentation)
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
)

model_base = TapasForQuestionAnswering.from_pretrained("google/tapas-base") # base (bert architecture)
model_wtq = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq", config=config) # leave as is
model_finetune = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq", config=config) # finetune
optimizer = AdamW(model_finetune.parameters(), lr=5e-5)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_bias', 'column_output_weights', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
def evaluate_model(model, dataloader, device):
    model.eval()
    total_loss = 0
    num_batches = 0
    with torch.no_grad():
        for batch in dataloader:
            # Move batch to device
            inputs = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**inputs)
            total_loss += outputs.loss.item()
            num_batches += 1
    return total_loss / num_batches if num_batches > 0 else float('inf')

import torch
from transformers import TapasForQuestionAnswering, TapasConfig, AdamW
import pandas as pd
from tqdm import tqdm

# Function to evaluate the model and calculate dev loss
def evaluate_model(model, dataloader, device):
    model.eval()
    total_loss = 0
    num_batches = 0
    with torch.no_grad():
        for batch in dataloader:
            # Move batch to device
            inputs = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**inputs)
            total_loss += outputs.loss.item()
            num_batches += 1
    return total_loss / num_batches if num_batches > 0 else float('inf')

def train_model(model, train_dataloader, optimizer, device, epochs, save_path):
    best_dev_loss = float('inf')
    model.to(device)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        total_loss = 0
        num_batches = 0

        # Training loop
        for batch in tqdm(train_dataloader, desc="Training"):
            optimizer.zero_grad()
            inputs = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            num_batches += 1

        avg_train_loss = total_loss / num_batches if num_batches > 0 else 0
        print(f"Average training loss: {avg_train_loss:.4f}")
        model.save_pretrained(save_path)
    
    print('Done Training')
    return model



In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fine-tune model
model_finetun = train_model(
    model=model_finetune,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    device=device,
    epochs=5,
    save_path="./wtq_finetune"
)

Epoch 1/5


Training:   0%|                                          | 0/12 [00:00<?, ?it/s]/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as posi

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
Training:  25%|████████▌                         | 3/12 [01:40<05:01, 33.53s/it]/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as posi

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

...

Average training loss: 0.8771


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Validation loss: 0.7453


'# finetune the WTQ model\nmodel_finetune.train()\nfor epoch in range(20):\n    print(\'On epoch:\', epoch) \n    for batch in train_dataloader:\n        input_ids = batch["input_ids"]\n        attention_mask = batch["attention_mask"]\n        token_type_ids = batch["token_type_ids"]\n        labels = batch["labels"]\n        numeric_values = batch["numeric_values"]\n        numeric_values_scale = batch["numeric_values_scale"]\n        float_answer = batch["float_answer"]\n        \n        if isinstance(numeric_values, pd.Series):\n            numeric_values = numeric_values.iloc[:] \n            \n        optimizer.zero_grad()\n\n        # forward + backward + optimize\n        outputs = model_finetune(\n            input_ids=input_ids,\n            attention_mask=attention_mask,\n            token_type_ids=token_type_ids,\n            labels=labels,\n            numeric_values=numeric_values,\n            numeric_values_scale=numeric_values_scale,\n            float_answer=float_ans

In [27]:
# Inference on Finetuned 
num_none = 0
num_agg = 0
num_correct = 0
num_correct_none = 0
num_correct_agg = 0
total_questions = 0

#saved_model_path = "wtq_finetune/"
#model_finetune = TapasForQuestionAnswering.from_pretrained(saved_model_path)
#tokenizer = TapasTokenizer.from_pretrained(saved_model_path)

for file in dev_datasets:
    print('#################### (Finetuned) dev on:', file,'####################')
    table = pd.read_csv(f'data/{file}').astype(str)
    dev_df = toy_df_dev[toy_df_dev['dataset'] == file]
    queries = list(dev_df['question'])
    inputs = tokenizer(
        table = table,
        queries = queries,
        padding="max_length",
        return_tensors="pt",
        truncation = True
    )

    outputs = model_finetune(**inputs)
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
    )

    # print results:
    id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

    answers = []
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(table.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(table.iat[coordinate])
            answers.append(", ".join(cell_values))

    i = 0
    real_answers = list(dev_df['sample_answer'])
    answer_type = list(dev_df['type'])
    for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
        total_questions += 1 
        print(query)
        print('Indeded type: ', answer_type[i])
        if predicted_agg == "NONE":
            print("     Predicted answer: " + answer)
            print("     Real answer: ", real_answers[i])
            if str(answer) == str(real_answers[i]):
                num_correct += 1
                num_correct_none += 1
            i += 1
            num_none += 1
            
        else:
            print("     Predicted Cells: " + predicted_agg + " > " + answer)
            if predicted_agg == 'COUNT':
                count = answer.split(',')
                answer = len(list(set(count)))
                print('     Predicted Count: ', answer)
                
            if predicted_agg == 'AVERAGE':
                total = 0
                a = answer.split(',')
                try:
                    a = [float(elem) for elem in a]
                except:
                    print('     Predicted Average: Error')
                    answer = None
                    continue
                num = len(a)
                answer = 0
                for elem in a:
                    answer += (elem/num)  
                print('     Predicted Average:', answer)
            if predicted_agg == 'SUM':
                s = answer.split(',')
                s = [float(elem) for elem in s]
                answer = 0
                for elem in s:
                    answer += elem
                print('     Predicted Sum:', answer)
                    
            print("     Real answer: ", str(real_answers[i]))
            if str(answer) == str(real_answers[i]):
                    num_correct += 1
                    num_correct_agg += 1
            i += 1
            num_agg += 1
            
print("")
print('Number of Questions: ', total_questions)
print('Overall Accuracy: ', num_correct/total_questions)
try:
    print('Number of NONE:', num_none)
    print('NONE Accuracy:', num_correct_none/num_none)
except:
    print('***** No questions with predicted Aggregation Tag NONE ***** ')
try:
    print('Number of Agg:', num_agg)
    print('Agg Accuracy:', num_correct_agg/num_agg)
except:
    print('***** No questions with predicted Aggregation Tag SUM, AVERAGE, or COUNT ***** ')
print("")

#################### (Finetuned) dev on: 010_ECommerce.csv ####################


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


What is the average age of the reviewers?
Indeded type:  number
     Predicted answer: 
     Real answer:  39.65
What's the highest number of positive feedback received for a review?
Indeded type:  number
     Predicted answer: 
     Real answer:  19
What is the most common rating given by reviewers?
Indeded type:  number
     Predicted answer: 
     Real answer:  5
How many unique clothing items are there in the dataset?
Indeded type:  number
     Predicted Cells: COUNT > 1095, 903, 830, 1047, 1110, 820, 1059, 1092, 22, 394, 1081, 1008, 1094, 1077, 1021, 1025, 834, 868, 829, 1078
     Predicted Count:  20
     Real answer:  20
Which department has the most reviews?
Indeded type:  category
     Predicted answer: 
     Real answer:  Dresses
Which class of clothing is most commonly reviewed?
Indeded type:  category
     Predicted answer: 
     Real answer:  Dresses
Which division is most commonly mentioned in the reviews?
Indeded type:  category
     Predicted answer: 
     Real answer: 

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


What's the total number of unique agencies handling complaints?
Indeded type:  number
     Predicted Cells: COUNT > HPD, NYPD, NYPD, DOB, DOHMH, DSNY, NYPD, DOB, HPD, NYPD, DOB, DEP, NYPD, NYPD, HPD, DPR, NYPD, NYPD, DEP, DSNY
     Predicted Count:  8
     Real answer:  7
How many complaints were raised at midnight?
Indeded type:  number
     Predicted Cells: COUNT > HEAT/HOT WATER, Illegal Parking, Noise - Commercial, Building/Use, Rodent, Snow, Abandoned Vehicle, Building/Use, WATER LEAK, Blocked Driveway, General Construction/Plumbing, Sewer, Noise - Street/Sidewalk, Derelict Vehicle, HEAT/HOT WATER, Maintenance or Facility, Noise - Residential, Noise - Residential, Air Quality, Sanitation Condition
     Predicted Count:  18
     Real answer:  2
How many unique descriptors are present in the dataset?
Indeded type:  number
     Predicted Cells: COUNT > APARTMENT ONLY, Commercial Overnight Parking, Loud Music/Party, Illegal Conversion Of Residential Building/Space, Condition Attractin

In [28]:
# Inference on WTQ 
num_none = 0
num_agg = 0
num_correct = 0
num_correct_none = 0
num_correct_agg = 0
total_questions = 0
for file in dev_datasets:
    print('#################### (WTQ) dev on:', file,'####################')
    table = pd.read_csv(f'data/{file}').astype(str)
    dev_df = toy_df_dev[toy_df_dev['dataset'] == file]
    queries = list(dev_df['question'])
    inputs = tokenizer(
        table = table,
        queries = queries,
        padding="max_length",
        return_tensors="pt",
        truncation = True
    )

    outputs = model_wtq(**inputs)
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
    )

    # print results:
    id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

    answers = []
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(table.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(table.iat[coordinate])
            answers.append(", ".join(cell_values))

    i = 0
    real_answers = list(dev_df['sample_answer'])
    answer_type = list(dev_df['type'])
    for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
        total_questions += 1 
        print(query)
        print('Indeded type: ', answer_type[i])
        if predicted_agg == "NONE":
            print("     Predicted answer: " + answer)
            print("     Real answer: ", real_answers[i])
            if str(answer) == str(real_answers[i]):
                num_correct += 1
                num_correct_none += 1
            i += 1
            num_none += 1
            
        else:
            print("     Predicted Cells: " + predicted_agg + " > " + answer)
            if predicted_agg == 'COUNT':
                count = answer.split(',')
                answer = len(list(set(count)))
                print('     Predicted Count: ', answer)
                
            if predicted_agg == 'AVERAGE':
                total = 0
                a = answer.split(',')
                try:
                    a = [float(elem) for elem in a]
                except:
                    print('     Predicted Average: Error')
                    answer = None
                    continue
                num = len(a)
                answer = 0
                for elem in a:
                    answer += (elem/num)  
                print('     Predicted Average:', answer)
            if predicted_agg == 'SUM':
                s = answer.split(',')
                s = [float(elem) for elem in s]
                answer = 0
                for elem in s:
                    answer += elem
                print('     Predicted Sum:', answer)
                    
            print("     Real answer: ", str(real_answers[i]))
            if str(answer) == str(real_answers[i]):
                    num_correct += 1
                    num_correct_agg += 1
            i += 1
            num_agg += 1
            
print("")
print('Number of Questions: ', total_questions)
print('Overall Accuracy: ', num_correct/total_questions)
try:
    print('Number of NONE:', num_none)
    print('NONE Accuracy:', num_correct_none/num_none)
except:
    print('***** No questions with predicted Aggregation Tag NONE ***** ')
try:
    print('Number of Agg:', num_agg)
    print('Agg Accuracy:', num_correct_agg/num_agg)
except:
    print('***** No questions with predicted Aggregation Tag SUM, AVERAGE, or COUNT ***** ')
print("")

#################### (WTQ) dev on: 010_ECommerce.csv ####################


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


What is the average age of the reviewers?
Indeded type:  number
     Predicted Cells: AVERAGE > 34, 57, 56, 36, 30, 36, 37, 39, 30, 30, 53, 36, 38, 31, 65, 22, 41, 33, 33, 56
     Predicted Average: 39.64999999999999
     Real answer:  39.65
What's the highest number of positive feedback received for a review?
Indeded type:  number
     Predicted Cells: AVERAGE > 5
     Predicted Average: 5.0
     Real answer:  19
What is the most common rating given by reviewers?
Indeded type:  number
     Predicted Cells: AVERAGE > 5
     Predicted Average: 5.0
     Real answer:  5
How many unique clothing items are there in the dataset?
Indeded type:  number
     Predicted Cells: COUNT > 1095, 903, 830, 1047, 1110, 820, 1059, 1092, 22, 394, 1081, 1008, 1094, 1077, 1021, 1025, 834, 868, 829, 1078
     Predicted Count:  20
     Real answer:  20
Which department has the most reviews?
Indeded type:  category
     Predicted answer: Dresses
     Real answer:  Dresses
Which class of clothing is most common

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


What's the total number of unique agencies handling complaints?
Indeded type:  number
     Predicted Cells: COUNT > HEAT/HOT WATER, Illegal Parking, Noise - Commercial, Building/Use, Rodent, Snow, Abandoned Vehicle, Building/Use, WATER LEAK, Blocked Driveway, General Construction/Plumbing, Sewer, Noise - Street/Sidewalk, Derelict Vehicle, HEAT/HOT WATER, Maintenance or Facility, Noise - Residential, Noise - Residential, Air Quality, Sanitation Condition
     Predicted Count:  18
     Real answer:  7
How many complaints were raised at midnight?
Indeded type:  number
     Predicted Cells: COUNT > HEAT/HOT WATER, Illegal Parking, Noise - Commercial, Building/Use, Rodent, Snow, Abandoned Vehicle, Building/Use, WATER LEAK, Blocked Driveway, General Construction/Plumbing, Sewer, Noise - Street/Sidewalk, Derelict Vehicle, HEAT/HOT WATER, Maintenance or Facility, Noise - Residential, Noise - Residential, Air Quality, Sanitation Condition
     Predicted Count:  18
     Real answer:  2
How many 

In [29]:
# Inference on Base 
num_none = 0
num_agg = 0
num_correct = 0
num_correct_none = 0
num_correct_agg = 0
total_questions = 0
for file in dev_datasets:
    print('#################### (Base) dev on:', file,'####################')
    table = pd.read_csv(f'data/{file}').astype(str)
    dev_df = toy_df_dev[toy_df_dev['dataset'] == file]
    queries = list(dev_df['question'])
    inputs = tokenizer(
        table = table,
        queries = queries,
        padding="max_length",
        return_tensors="pt",
        truncation = True
    )

    outputs = model_base(**inputs)
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
    )

    # print results:
    id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

    answers = []
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(table.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(table.iat[coordinate])
            answers.append(", ".join(cell_values))

    i = 0
    real_answers = list(dev_df['sample_answer'])
    answer_type = list(dev_df['type'])
    for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
        total_questions += 1 
        print(query)
        print('Indeded type: ', answer_type[i])
        if predicted_agg == "NONE":
            print("     Predicted answer: " + answer)
            print("     Real answer: ", real_answers[i])
            if str(answer) == str(real_answers[i]):
                num_correct += 1
                num_correct_none += 1
            i += 1
            num_none += 1
            
        else:
            print("     Predicted Cells: " + predicted_agg + " > " + answer)
            if predicted_agg == 'COUNT':
                count = answer.split(',')
                answer = len(list(set(count)))
                print('     Predicted Count: ', answer)
                
            if predicted_agg == 'AVERAGE':
                total = 0
                a = answer.split(',')
                try:
                    a = [float(elem) for elem in a]
                except:
                    print('     Predicted Average: Error')
                    answer = None
                    continue
                num = len(a)
                answer = 0
                for elem in a:
                    answer += (elem/num)  
                print('     Predicted Average:', answer)
            if predicted_agg == 'SUM':
                s = answer.split(',')
                s = [float(elem) for elem in s]
                answer = 0
                for elem in s:
                    answer += elem
                print('     Predicted Sum:', answer)
                    
            print("     Real answer: ", str(real_answers[i]))
            if str(answer) == str(real_answers[i]):
                    num_correct += 1
                    num_correct_agg += 1
            i += 1
            num_agg += 1
            
print("")
print('Number of Questions: ', total_questions)
print('Overall Accuracy: ', num_correct/total_questions)
try:
    print('Number of NONE:', num_none)
    print('NONE Accuracy:', num_correct_none/num_none)
except:
    print('***** No questions with predicted Aggregation Tag NONE ***** ')
try:
    print('Number of Agg:', num_agg)
    print('Agg Accuracy:', num_correct_agg/num_agg)
except:
    print('***** No questions with predicted Aggregation Tag SUM, AVERAGE, or COUNT ***** ')
print("")

#################### (Base) dev on: 010_ECommerce.csv ####################


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


ValueError: Expected parameter logits (Tensor of shape (8, 2048)) of distribution Bernoulli(logits: torch.Size([8, 2048])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan,  ..., 0., 0., 0.],
        [nan, nan, nan,  ..., 0., 0., 0.],
        [nan, nan, nan,  ..., 0., 0., 0.],
        ...,
        [nan, nan, nan,  ..., 0., 0., 0.],
        [nan, nan, nan,  ..., 0., 0., 0.],
        [nan, nan, nan,  ..., 0., 0., 0.]], grad_fn=<ViewBackward0>)

Report accuracy train and dev (x-axis epoch, and y-axis loss) -> jsut show that I am thinking about overfitting
Run error analysis on answer length by accuracy 
fix and and train data split
talk about if I had more time... this is what I would have done